**Air Quality Reports Per Neighborhood in Pittsburgh**

For this ranking, air quality will be used to determine the most environmentally friendly neighborhood in Pittsburgh. As air quality diminishes, levels of pollutants in the air will be present in greater concentrations, leading to poorer quality of life for humans, plants, and animals. The neighborhood with the lowest concentration of pollutants in parts per billion will be considered the best.\
\
*Note: Neighborhoods with inadequate report values have not been included.*

In [30]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plot

**Before working with the data, unnecessary units and locations must be removed from consideration**

In [2]:
air = pd.read_csv("air_quality.csv", index_col= "_id")

air.replace(to_replace="Lawrenceville 2", value="Lawrenceville", inplace= True)
air['site'].replace({'Liberty 2' : 'Liberty'})
air['site'].replace({'South Fayette' : 'Allegheny'}) # also N. Braddock and Avalon and Harrison Twp.
air['site'].replace({'Courthouse' : 'Downtown'}) # ?


air.head() # replace works

,datetime_est,site,parameter,is_valid,report_value,unit,unit_description,highest_flag,aqs_parameter_category
_id,,,,,,,,,
1,2016-01-01T00:00:00,Lincoln,PM25,t,3.000,UG/M3,micrograms per cubic meter,NaN,Particulate
2,2016-01-01T00:00:00,Lincoln,PER_F2,t,67.000,PERCENT,Percent,NaN,NaN
3,2016-01-01T00:00:00,Liberty,SO2,t,0.001,PPM,Parts Per Million,C,Criteria Pollutants
4,2016-01-01T00:00:00,Liberty,SONICWS,t,6.900,MPH,miles per hour,NaN,Meteorological
5,2016-01-01T00:00:00,Liberty,SONICWD,t,259.000,DEG,Degrees,NaN,Meteorological


In [4]:
# CONVERSION HELPERS

def ppm_to_ppb(df):
    if(df['unit'] == 'PPM'):
        return df['report_value']*1000
    else:
        return df['report_value']

def ppm_to_ppb_label(df):
    if(df['unit'] == 'PPM'):
        return 'PPB'
    else:
        return df['unit']
    
def ppm_to_ppb_desc(df):
    if(df['unit_description'] == 'Parts Per Million'):
        return 'Parts Per Billion'
    else:
        return df['unit_description']

In [5]:
# CONVERTING UNITS
ppm = air[(air['unit'] == 'PPM')]
ppb = air[(air['unit'] == 'PPB')]

ppm['report_value'] = ppm.apply(ppm_to_ppb, axis=1)
ppm['unit'] = ppm.apply(ppm_to_ppb_label, axis=1)
ppm['unit_description'] = ppm.apply(ppm_to_ppb_desc, axis=1)

# 1ppm = 1000ppb

<ipython-input-5-82bf32cbcea2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppm['report_value'] = ppm.apply(ppm_to_ppb, axis=1)
<ipython-input-5-82bf32cbcea2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppm['unit'] = ppm.apply(ppm_to_ppb_label, axis=1)
<ipython-input-5-82bf32cbcea2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,datetime_est,site,parameter,is_valid,report_value,unit,unit_description,highest_flag,aqs_parameter_category
_id,,,,,,,,,
3,2016-01-01T00:00:00,Liberty,SO2,t,1.0,PPB,Parts Per Billion,C,Criteria Pollutants
6,2016-01-01T00:00:00,Liberty,H2S,t,0.0,PPB,Parts Per Billion,C,NaN
24,2016-01-01T00:00:00,North Braddock,SO2,t,0.0,PPB,Parts Per Billion,C,Criteria Pollutants
27,2016-01-01T00:00:00,Liberty,H2S_SO2,t,0.0,PPB,Parts Per Billion,C,NaN
44,2016-01-01T00:00:00,Harrison Township,OZONE,t,27.0,PPB,Parts Per Billion,NaN,Criteria Pollutants


In [6]:
# MAKE CSVs  -->  RUN ONCE

ppm.to_csv('ppm.csv')
ppb.to_csv('ppb.csv')

In [9]:
# COMBINE CSVs  -->  RUN ONCE

df = pd.concat( 
    map(pd.read_csv, ['ppm.csv', 'ppb.csv'])) 

df.tail()

df.to_csv('ppm_ppb.csv')

df['site'].unique()

array(['Liberty', 'North Braddock', 'Harrison Township', 'Lawrenceville',
       'South Fayette', 'Avalon', 'Court House', 'Parkway East',
       'Flag Plaza'], dtype=object)

In [12]:
# NEIGHBORS INFO COLLECT

data = pd.read_csv("ppm_ppb.csv", index_col="_id")

neighbor_qual = {}

In [18]:
def average(neighbor, data):
    # avg = df[df['site'] == 'name']['report_value'].mean()
    
    avg_qual = data[data['site'] == neighbor]['report_value'].mean()
    return avg_qual

def load_neighbors(df, neighbor_qual):
    neighbors = df['site'].unique()
    
    for hood in neighbors:
        # add to dict
        neighbor_qual[hood] = average(hood, df)



In [19]:
load_neighbors(data, neighbor_qual)

In [24]:
min_val = data['report_value'].min()
max_val = data['report_value'].max()
mid_val = (max_val-min_val)/2

print(neighbor_qual)

2273.05
{'Liberty': 3.9574790765601304, 'North Braddock': 1.9576470588235295, 'Harrison Township': 14.275813395440752, 'Lawrenceville': 48.070802489691204, 'South Fayette': 22.80795012736292, 'Avalon': 0.4820893659510734, 'Court House': 2.3984507840544116, 'Parkway East': 126.14351781989224, 'Flag Plaza': 0.27454545454545887}


In [35]:
vals = data['site'].value_counts()

plot.rcParams.update({'font.size': 100})

pd.Series(vals).plot(kind='barh', rot=0.5, edgecolor='black', figsize=(100, 300))

plot.xlabel('Air Quality (PPB)')  
plot.ylabel('Neighborhood')
plot.title('Air Quality Reports Per Pittsburgh Neighborhood')  

for i, val in enumerate(vals):
    plot.text(val, i, str(val), ha='left', va='center')

**Normalizing the data to a standardized scale of 1 to 10**

In [34]:
scores = 1 + (((vals - vals.min()) * (10 - 1)) / (vals.max() - vals.min()))
pd.DataFrame({'site': scores.index, 'score': scores},  index=vals.index)

,site,score
Lawrenceville,Lawrenceville,10.000000
Harrison Township,Harrison Township,7.306114
Parkway East,Parkway East,5.581110
Avalon,Avalon,4.109051
Liberty,Liberty,4.108311
South Fayette,South Fayette,4.108311
North Braddock,North Braddock,1.162713
Flag Plaza,Flag Plaza,1.162713
Court House,Court House,1.000000


**Verdict** \
Lawrenceville is the least environmentally friendly neighborhood in terms of air quality, as it has the highest level concentration (Parts Per Billion) of pollutants. The Courthouse (Downtown), Flag Plaza, and North Braddock all scored for some of the best neighborhoods in terms of air quality.